# gpkg2csv

Przygotowanie danych:
 - odczyt z pliku `gpkg/shp`
 - zapis wybranych kolumn jako `csv`

In [1]:
import fiona
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
pd.options.display.width = 250
np.set_printoptions(linewidth=250)

In [3]:
%ls ../testy/

cros01.csv  dane01.gpkg   r02_trueFalse.csv   r04_modern1.csv  raport.html
dane01.csv  r01_cros.csv  r03_classicAcc.csv  r05_modern2.csv


### Adresy plików
- do otwarcia
- do zapisu wyników

In [4]:
# adres pliku z danymi wejściowymi 'gpkg/shp'
adGpkg = Path('../testy/dane01.gpkg').resolve()

# adres pliku z wybranymi kolumnami
adDf = adGpkg.with_suffix('.csv')

# adres do zapisania cross matrix
adCros = Path('../testy/cros01.csv').resolve()

# adres pliku z etykietami referencyjnymi
adLabels = Path('./testy/labels01.json').resolve().as_posix()

print(f'{adGpkg}\n{adDf}\n{adCros}\n{adLabels}')

/home/u1/03_Programowanie/03Python/skrypty/accuracy/testy/dane01.gpkg
/home/u1/03_Programowanie/03Python/skrypty/accuracy/testy/dane01.csv
/home/u1/03_Programowanie/03Python/skrypty/accuracy/testy/cros01.csv
/home/u1/03_Programowanie/03Python/skrypty/accuracy/jupyter/testy/labels01.json


# 1. Przegląd danych pliku gpkg

In [5]:
with fiona.open(adGpkg.as_posix(),encoding='Windows-1250') as src:
    print(src[1]['properties'].keys(),'\n\n')
    print(src[1]['properties']['roslinaFul'],'\n\n')
    rosl = []
    for it in src:
        #print(it['properties'].keys(),'\n')
        rosl.append(it['properties']['roslinaFul'])
        
print(set(rosl))

odict_keys(['fid', 'obszar', 'nr_produce', 'dzrol', 'powierzchn', 'grupa_upra', 'roslina', 'gid', 'roslinaNow', 'roslinaFul', '_ML_20medi', '_ML_20majo']) 


kukurydza 


{'ĹĽyto ozime', 'pszenĹĽyto ozime', 'TUZ', 'pszenica jara', 'kukurydza', 'ziemniak', 'rzepik ozimy', 'owies', 'jÄ™czmieĹ„ jary', 'jÄ™czmieĹ„ ozimy', 'lucerna', 'rzepak ozimy', 'burak cuk', 'pszenica ozima', 'trawy'}


# 2. DataFRame
Pobranie danych z pliku `gpkg` - zapis do pd.DataFrame


In [6]:
gid,rosl,ml1,ml2 = [],[],[],[]

with fiona.open(adGpkg.as_posix(),encoding='Windows-1250') as src:
    for it in src:
        #gid.append(it['properties'].get('gid'))
        rosl.append(it['properties'].get('roslinaFul'))
        #ml1.append(it['properties'].get('_ML_20medi'))
        ml2.append(int(it['properties'].get('_ML_20majo')))


In [7]:
#df = pd.DataFrame({'gid':gid,'roslFul':rosl, 'ML20majo':ml2})
df = pd.DataFrame({'roslFul':rosl, 'ML20majo':ml2})
df.columns = ['nazwa','predict']
df.head()

,nazwa,predict
0,kukurydza,4
1,kukurydza,4
2,kukurydza,4
3,kukurydza,4
4,kukurydza,7


In [8]:
# pokaz unikalne wartości kolumn
for col in df.columns:
    print(f'{col}:\n{pd.unique(df.loc[:,col])}\n')

nazwa:
['kukurydza' 'burak cuk' 'rzepak ozimy' 'jÄ™czmieĹ„ ozimy' 'lucerna' 'trawy' 'pszenica ozima' 'TUZ' 'ĹĽyto ozime' 'pszenica jara' 'rzepik ozimy' 'pszenĹĽyto ozime' 'jÄ™czmieĹ„ jary' 'ziemniak' 'owies']

predict:
[ 4  7  0  9 12  5  1 13  3  8  2  6 14 11 10]



# 3. Kodowanie polskich znaków
Poprawa nazw z polskimi literami

In [9]:
# get kolumn nazwa
old = pd.unique(df.loc[:,'nazwa']).tolist()
print(old)

['kukurydza', 'burak cuk', 'rzepak ozimy', 'jÄ™czmieĹ„ ozimy', 'lucerna', 'trawy', 'pszenica ozima', 'TUZ', 'ĹĽyto ozime', 'pszenica jara', 'rzepik ozimy', 'pszenĹĽyto ozime', 'jÄ™czmieĹ„ jary', 'ziemniak', 'owies']


In [10]:
new = ['kukurydza','burakC','rzepakO','jeczmienO','lucerna','trawy','pszenicaO','TUZ','zytoO','pszenicaJ',
 'rzepikO','pszenzytoO','jeczmienJ','ziemniak','owies']
print(new)

['kukurydza', 'burakC', 'rzepakO', 'jeczmienO', 'lucerna', 'trawy', 'pszenicaO', 'TUZ', 'zytoO', 'pszenicaJ', 'rzepikO', 'pszenzytoO', 'jeczmienJ', 'ziemniak', 'owies']


In [11]:
for o,n in zip(old,new):
    idx = df.loc[:,'nazwa']==o
    df.loc[idx,'nazwa'] = n

In [12]:
# sprawdzenie
pd.unique(df.loc[:,'nazwa'])

array(['kukurydza', 'burakC', 'rzepakO', 'jeczmienO', 'lucerna', 'trawy', 'pszenicaO', 'TUZ', 'zytoO', 'pszenicaJ', 'rzepikO', 'pszenzytoO', 'jeczmienJ', 'ziemniak', 'owies'], dtype=object)

In [13]:
df

,nazwa,predict
0,kukurydza,4
1,kukurydza,4
2,kukurydza,4
3,kukurydza,4
4,kukurydza,7
...,...,...
2185,owies,5
2186,owies,10
2187,owies,0
2188,owies,12


# 4. Zapis danych do pliku

In [14]:
# zapisz dane jako csv
adDf = adGpkg.with_suffix('.csv')
df.to_csv(adDf,sep=';',index=False)
print(f'Zapis do pliku:\n\t{adDf}')

Zapis do pliku:
	/home/u1/03_Programowanie/03Python/skrypty/accuracy/testy/dane01.csv


# 5. Mapowanie etykiet referencyjnych do nowej kolumny

- każdej nazwie rośliny przypisz etykietę liczbową lub znakową - `etykieta referencyjna`
- zapisz te dane jako słownik - plik `json`

### etykiety ręcznie podane i zapisane jako plik `json`
```python
labels = {'kukurydza':0,'pszenica jara':1, 'rzepik ozimy':2, 'pszenzyto ozime':3, 'jeczmien jary':4,\
         'ziemniak':5, 'owies':6, 'burak cukrowy':7, 'rzepak ozimy':8, 'jeczmien ozimy':9, 'lucerna':10, \
         'trawy':11, 'pszenica ozima':12, 'TUZ': 13, 'zyto ozime':14 }```

In [15]:
# ustala referencyjne etykiety liczbowe 
kod = list(range(len(new)))
labels = dict(zip(new,kod))
labels

{'kukurydza': 0,
 'burakC': 1,
 'rzepakO': 2,
 'jeczmienO': 3,
 'lucerna': 4,
 'trawy': 5,
 'pszenicaO': 6,
 'TUZ': 7,
 'zytoO': 8,
 'pszenicaJ': 9,
 'rzepikO': 10,
 'pszenzytoO': 11,
 'jeczmienJ': 12,
 'ziemniak': 13,
 'owies': 14}

In [16]:
for nazwa,kod in zip(new,kod):
    idx = df.loc[:,'nazwa']==nazwa
    df.loc[idx,'referencja'] = kod

df = df.astype({'referencja':int})

# zmiana nazw kolumn i ich kolejności, na zgodne z innymi skryptami
kol1 = ['short','predict','true']
kol2 = ['short','true','predict']

df.columns = kol1
df = df.loc[:,kol2]
df

,short,true,predict
0,kukurydza,0,4
1,kukurydza,0,4
2,kukurydza,0,4
3,kukurydza,0,4
4,kukurydza,0,7
...,...,...,...
2185,owies,14,5
2186,owies,14,10
2187,owies,14,0
2188,owies,14,12


In [17]:
# zapisz dane jako csv
#adDf = adGpkg.with_suffix('.csv')
df.to_csv(adDf,sep=';',index=False)

# cross matrix

### 1. Układ macierzy
  >- każda kolumna zawiera wyniki klasyfikacji czyli etykiety 'przewidywane' (`predicted`)
  >- wiersze zawierają dane referencyjne czyli etykiety służące sprawdzeniu poprawności klasyfikacji
  
### 2. Sprawdzenie na danych próbnych układu kolumn

In [18]:
refTest = pd.Series(['Adam','Ewa','Marek'])
predictTest = pd.Series(['men','kob','men'])
print(refTest,'\n\n',predictTest)

0     Adam
1      Ewa
2    Marek
dtype: object 

 0    men
1    kob
2    men
dtype: object


In [19]:
pd.crosstab(refTest,predictTest,rownames=['referencje'],colnames=['predicted'])

predicted,kob,men
referencje,,
Adam,0,1
Ewa,1,0
Marek,0,1


### 3. Cross
 - utworzenie nowych, skróconych nazw roślin - żeby mieściły się w kolumnach tabeli

In [20]:
#df = pd.read_csv('/media/u1/1674-9C99/acc/ndviClipped_ready.csv',sep=';')
df

,short,true,predict
0,kukurydza,0,4
1,kukurydza,0,4
2,kukurydza,0,4
3,kukurydza,0,4
4,kukurydza,0,7
...,...,...,...
2185,owies,14,5
2186,owies,14,10
2187,owies,14,0
2188,owies,14,12


In [21]:
cros = pd.crosstab(df.true,df.predict,rownames=['true'],colnames=['predicted'])
cros

predicted,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
true,,,,,,,,,,,,,,,
0,67,19,6,36,100,55,11,10,3,23,0,1,44,26,1
1,12,8,3,19,2,21,5,9,3,12,1,0,18,7,0
2,2,4,34,15,0,43,0,3,11,1,0,0,6,1,1
3,2,2,1,8,2,5,0,2,0,62,0,1,55,0,7
4,2,18,1,2,1,10,0,3,0,2,2,0,1,11,0
5,2,13,0,1,1,1,0,1,0,2,1,4,6,22,0
6,20,9,6,13,32,11,11,1,2,96,0,4,75,8,0
7,2,35,0,0,0,0,1,4,0,0,0,0,1,248,0
8,3,2,0,10,2,1,2,2,0,33,0,0,52,4,6


In [22]:
# na podstawie kolumn 'short' i 'true' tworzy słownik: {short:true}
nazwy = pd.unique(df.short)
print(f'nazwy:\n\t{nazwy}\n\n')
kod = []
for n in nazwy:
    idx = df.loc[:,'short'] == n
    val = pd.unique(df.loc[idx,'true'])[0]
    #print(val)
    kod.append(val)
    

mapaNazw = dict((zip(kod,nazwy)))
print(f'mapaNazw:\n\t{mapaNazw}')

nazwy:
	['kukurydza' 'burakC' 'rzepakO' 'jeczmienO' 'lucerna' 'trawy' 'pszenicaO' 'TUZ' 'zytoO' 'pszenicaJ' 'rzepikO' 'pszenzytoO' 'jeczmienJ' 'ziemniak' 'owies']


mapaNazw:
	{0: 'kukurydza', 1: 'burakC', 2: 'rzepakO', 3: 'jeczmienO', 4: 'lucerna', 5: 'trawy', 6: 'pszenicaO', 7: 'TUZ', 8: 'zytoO', 9: 'pszenicaJ', 10: 'rzepikO', 11: 'pszenzytoO', 12: 'jeczmienJ', 13: 'ziemniak', 14: 'owies'}


In [23]:
# na podstawie 'mapaNazw' tworzy indeks nazw dla cross matrix
newIndex = []
tmp = cros.columns.values.tolist()
print(f'tmp:  {tmp}\n')

for i in tmp:
    newIndex.append(mapaNazw[i])

#for i,k in enumerate(mapaNazw.keys()):
 #   print(i,k)
  #  if tmp[i] == k:
   #     newIndex.append(mapaNazw.get(k))

newIndex

tmp:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]



['kukurydza',
 'burakC',
 'rzepakO',
 'jeczmienO',
 'lucerna',
 'trawy',
 'pszenicaO',
 'TUZ',
 'zytoO',
 'pszenicaJ',
 'rzepikO',
 'pszenzytoO',
 'jeczmienJ',
 'ziemniak',
 'owies']

In [24]:
# ustaw nazwy kolumn i wierszy
cros.columns = newIndex
cros.axes[1].name='predicted'

cros.index = newIndex
cros.axes[0].name='true'

cros

predicted,kukurydza,burakC,rzepakO,jeczmienO,lucerna,trawy,pszenicaO,TUZ,zytoO,pszenicaJ,rzepikO,pszenzytoO,jeczmienJ,ziemniak,owies
true,,,,,,,,,,,,,,,
kukurydza,67,19,6,36,100,55,11,10,3,23,0,1,44,26,1
burakC,12,8,3,19,2,21,5,9,3,12,1,0,18,7,0
rzepakO,2,4,34,15,0,43,0,3,11,1,0,0,6,1,1
jeczmienO,2,2,1,8,2,5,0,2,0,62,0,1,55,0,7
lucerna,2,18,1,2,1,10,0,3,0,2,2,0,1,11,0
trawy,2,13,0,1,1,1,0,1,0,2,1,4,6,22,0
pszenicaO,20,9,6,13,32,11,11,1,2,96,0,4,75,8,0
TUZ,2,35,0,0,0,0,1,4,0,0,0,0,1,248,0
zytoO,3,2,0,10,2,1,2,2,0,33,0,0,52,4,6


In [25]:
sum1 = cros.sum(axis=0)
cros.loc['TotalKol',:] = sum1
sum1

predicted
kukurydza     141
burakC        128
rzepakO        58
jeczmienO     140
lucerna       213
trawy         183
pszenicaO      45
TUZ            43
zytoO          21
pszenicaJ     390
rzepikO         5
pszenzytoO     12
jeczmienJ     436
ziemniak      339
owies          36
dtype: int64

In [26]:
sum2 = cros.sum(axis=1)
cros.loc[:,'TotalRow'] = sum2
sum2

true
kukurydza      402.0
burakC         120.0
rzepakO        121.0
jeczmienO      147.0
lucerna         53.0
trawy           54.0
pszenicaO      288.0
TUZ            291.0
zytoO          117.0
pszenicaJ        8.0
rzepikO          3.0
pszenzytoO     402.0
jeczmienJ      141.0
ziemniak        11.0
owies           32.0
TotalKol      2190.0
dtype: float64

In [27]:
cros

predicted,kukurydza,burakC,rzepakO,jeczmienO,lucerna,trawy,pszenicaO,TUZ,zytoO,pszenicaJ,rzepikO,pszenzytoO,jeczmienJ,ziemniak,owies,TotalRow
true,,,,,,,,,,,,,,,,
kukurydza,67.0,19.0,6.0,36.0,100.0,55.0,11.0,10.0,3.0,23.0,0.0,1.0,44.0,26.0,1.0,402.0
burakC,12.0,8.0,3.0,19.0,2.0,21.0,5.0,9.0,3.0,12.0,1.0,0.0,18.0,7.0,0.0,120.0
rzepakO,2.0,4.0,34.0,15.0,0.0,43.0,0.0,3.0,11.0,1.0,0.0,0.0,6.0,1.0,1.0,121.0
jeczmienO,2.0,2.0,1.0,8.0,2.0,5.0,0.0,2.0,0.0,62.0,0.0,1.0,55.0,0.0,7.0,147.0
lucerna,2.0,18.0,1.0,2.0,1.0,10.0,0.0,3.0,0.0,2.0,2.0,0.0,1.0,11.0,0.0,53.0
trawy,2.0,13.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,1.0,4.0,6.0,22.0,0.0,54.0
pszenicaO,20.0,9.0,6.0,13.0,32.0,11.0,11.0,1.0,2.0,96.0,0.0,4.0,75.0,8.0,0.0,288.0
TUZ,2.0,35.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,248.0,0.0,291.0
zytoO,3.0,2.0,0.0,10.0,2.0,1.0,2.0,2.0,0.0,33.0,0.0,0.0,52.0,4.0,6.0,117.0


### 4. Zapisz cros matrix do `csv`
>  `adCros = Path('cros1.csv').resolve()`

In [28]:
%ls

accuracy.ipynb  gpkg2csv.ipynb


In [3]:
ax = Path('/media/u1/1674-9C99/acc/shape/czempinNew.shp').resolve()
with fiona.open(ax.as_posix()) as src:
    print(src[1]['properties'].keys(),'\n\n')
    print(src[1]['properties']['roslinaFul'],'\n\n')
    rosl = []
    for it in src:
        #print(it['properties'].keys(),'\n')
        rosl.append(it['properties']['roslinaFul'])

odict_keys(['fid', 'obszar', 'nr_produce', 'dzrol', 'powierzchn', 'grupa_upra', 'roslina', 'gid', 'roslinaNow', 'roslinaFul', 'short', 'true']) 


TUZ 


